In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import (StandardScaler, OrdinalEncoder,LabelEncoder, MinMaxScaler, OneHotEncoder)
from sklearn.preprocessing import Normalizer, MaxAbsScaler , RobustScaler, PowerTransformer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import copy
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
df1 = pd.read_parquet('/content/drive/MyDrive/Datasets/UNSW-NB15/UNSW_NB15_training-set.parquet')
df2 = pd.read_parquet('/content/drive/MyDrive/Datasets/UNSW-NB15/UNSW_NB15_testing-set.parquet')

In [ ]:
train_data = result = pd.concat([df1,df2], axis=0)

In [ ]:
train_data

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,attack_cat,label
0,0.000011,udp,-,INT,2,0,496,0,90909.093750,1.803636e+08,...,0,0,1,1,0,0,0,0,Normal,0
1,0.000008,udp,-,INT,2,0,1762,0,125000.000000,8.810000e+08,...,0,0,1,1,0,0,0,0,Normal,0
2,0.000005,udp,-,INT,2,0,1068,0,200000.000000,8.544000e+08,...,0,0,1,1,0,0,0,0,Normal,0
3,0.000006,udp,-,INT,2,0,900,0,166666.656250,6.000000e+08,...,0,0,2,1,0,0,0,0,Normal,0
4,0.000010,udp,-,INT,2,0,2126,0,100000.000000,8.504000e+08,...,0,0,2,1,0,0,0,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,0.000009,udp,dns,INT,2,0,114,0,111111.109375,5.066666e+07,...,0,0,24,13,0,0,0,0,Generic,1
175337,0.505762,tcp,-,FIN,10,8,620,354,33.612648,8.826286e+03,...,0,0,1,1,0,0,0,0,Shellcode,1
175338,0.000009,udp,dns,INT,2,0,114,0,111111.109375,5.066666e+07,...,0,0,3,3,0,0,0,0,Generic,1
175339,0.000009,udp,dns,INT,2,0,114,0,111111.109375,5.066666e+07,...,0,0,30,14,0,0,0,0,Generic,1


In [ ]:
train_data = train_data.drop_duplicates()
train_data = train_data.dropna()
train_data = train_data.replace([np.inf, -np.inf], np.nan)
train_data =train_data.drop("label", axis=1)
train_data.rename(columns={'attack_cat': 'label'}, inplace=True)

In [ ]:
train_data

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,dmean,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,label
0,0.000011,udp,-,INT,2,0,496,0,90909.093750,1.803636e+08,...,0,0,0,1,1,0,0,0,0,Normal
1,0.000008,udp,-,INT,2,0,1762,0,125000.000000,8.810000e+08,...,0,0,0,1,1,0,0,0,0,Normal
2,0.000005,udp,-,INT,2,0,1068,0,200000.000000,8.544000e+08,...,0,0,0,1,1,0,0,0,0,Normal
3,0.000006,udp,-,INT,2,0,900,0,166666.656250,6.000000e+08,...,0,0,0,2,1,0,0,0,0,Normal
4,0.000010,udp,-,INT,2,0,2126,0,100000.000000,8.504000e+08,...,0,0,0,2,1,0,0,0,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175267,1.914309,tcp,smtp,FIN,74,30,69997,2132,53.805313,2.885720e+05,...,71,0,0,1,1,0,0,0,0,Exploits
175277,3.719110,tcp,-,FIN,66,340,3086,426483,108.897018,6.539199e+03,...,1254,0,0,1,1,0,0,0,0,Exploits
175299,0.996503,tcp,pop3,FIN,20,30,942,16684,49.171955,7.185126e+03,...,556,0,0,1,1,0,0,0,0,Exploits
175305,1.557125,tcp,smtp,FIN,28,22,12601,1954,31.468250,6.242787e+04,...,89,0,0,1,1,0,0,0,0,Exploits


In [ ]:
# majority class

Normal = train_data[train_data['label'] == 'Normal']
Exploits = train_data[train_data['label'] == 'Exploits']
Fuzzers = train_data[train_data['label'] == 'Fuzzers']


# minority class

Reconnaissance = train_data[train_data['label'] == 'Reconnaissance']
DoS = train_data[train_data['label'] == 'DoS']
Generic = train_data[train_data['label'] == 'Generic']
Analysis = train_data[train_data['label'] == 'Analysis']
Backdoor = train_data[train_data['label'] == 'Backdoor']
Shellcode = train_data[train_data['label'] == 'Shellcode']
Worms = train_data[train_data['label'] == 'Worms']

In [ ]:
majority_class_data = pd.concat([Normal, Exploits, Fuzzers], axis=0)

In [ ]:
majority_class_data['label'].value_counts()

Normal            79965
Exploits          26749
Fuzzers           18313
Analysis              0
Backdoor              0
DoS                   0
Generic               0
Reconnaissance        0
Shellcode             0
Worms                 0
Name: label, dtype: int64

In [ ]:
majority_class_data.reset_index(drop=True, inplace=True)

In [ ]:
majority_class_data

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,dmean,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,label
0,0.000011,udp,-,INT,2,0,496,0,90909.093750,180363632.0,...,0,0,0,1,1,0,0,0,0,Normal
1,0.000008,udp,-,INT,2,0,1762,0,125000.000000,881000000.0,...,0,0,0,1,1,0,0,0,0,Normal
2,0.000005,udp,-,INT,2,0,1068,0,200000.000000,854400000.0,...,0,0,0,1,1,0,0,0,0,Normal
3,0.000006,udp,-,INT,2,0,900,0,166666.656250,600000000.0,...,0,0,0,2,1,0,0,0,0,Normal
4,0.000010,udp,-,INT,2,0,2126,0,100000.000000,850400000.0,...,0,0,0,2,1,0,0,0,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125022,0.000003,udp,-,INT,2,0,160,0,333333.312500,213333328.0,...,0,0,0,1,1,0,0,0,0,Fuzzers
125023,0.000009,udp,-,INT,2,0,182,0,111111.109375,80888888.0,...,0,0,0,1,1,0,0,0,0,Fuzzers
125024,0.000008,udp,-,INT,2,0,222,0,125000.000000,111000000.0,...,0,0,0,3,1,0,0,0,0,Fuzzers
125025,0.000002,udp,-,INT,2,0,208,0,500000.000000,416000000.0,...,0,0,0,2,1,0,0,0,0,Fuzzers


In [ ]:
majority_class_data.to_csv('/content/sample_data/majority_class.csv', index=False)

In [ ]:
majority_class_data = pd.read_csv('/content/majority_class.csv')

In [ ]:
majority_class_data

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,dmean,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,label
0,0.000011,udp,-,INT,2,0,496,0,90909.09,180363630.0,...,0,0,0,1,1,0,0,0,0,Normal
1,0.000008,udp,-,INT,2,0,1762,0,125000.00,881000000.0,...,0,0,0,1,1,0,0,0,0,Normal
2,0.000005,udp,-,INT,2,0,1068,0,200000.00,854400000.0,...,0,0,0,1,1,0,0,0,0,Normal
3,0.000006,udp,-,INT,2,0,900,0,166666.66,600000000.0,...,0,0,0,2,1,0,0,0,0,Normal
4,0.000010,udp,-,INT,2,0,2126,0,100000.00,850400000.0,...,0,0,0,2,1,0,0,0,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125022,0.000003,udp,-,INT,2,0,160,0,333333.30,213333330.0,...,0,0,0,1,1,0,0,0,0,Fuzzers
125023,0.000009,udp,-,INT,2,0,182,0,111111.11,80888890.0,...,0,0,0,1,1,0,0,0,0,Fuzzers
125024,0.000008,udp,-,INT,2,0,222,0,125000.00,111000000.0,...,0,0,0,3,1,0,0,0,0,Fuzzers
125025,0.000002,udp,-,INT,2,0,208,0,500000.00,416000000.0,...,0,0,0,2,1,0,0,0,0,Fuzzers


In [ ]:
from sklearn.utils import shuffle

# Assuming df is your DataFrame
majority_class_data = shuffle(majority_class_data)

# If you want to reset the index after shuffling
majority_class_data.reset_index(drop=True, inplace=True)

In [ ]:
majority_data_subset = copy.deepcopy(majority_class_data)

In [ ]:
majority_data_subset

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,dmean,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,label
0,0.059893,tcp,-,FIN,60,62,3614,50036,2020.269400,4.747132e+05,...,807,0,0,1,1,0,0,0,0,Normal
1,0.002852,tcp,ftp-data,FIN,6,8,320,1826,4558.204600,7.489481e+05,...,228,0,0,1,1,0,0,0,0,Normal
2,1.195022,tcp,http,FIN,14,18,1684,10168,25.940945,1.047010e+04,...,565,1,3924,1,1,0,0,1,0,Normal
3,0.015786,tcp,-,FIN,48,50,2974,30506,6144.685500,1.476245e+06,...,610,0,0,1,1,0,0,0,0,Normal
4,0.024833,tcp,-,FIN,60,62,3598,48918,4872.549000,1.140096e+06,...,789,0,0,1,1,0,0,0,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125022,0.019016,tcp,-,FIN,48,50,2958,32642,5100.968000,1.218763e+06,...,653,0,0,1,1,0,0,0,0,Normal
125023,0.788817,tcp,-,FIN,122,126,7818,15800,313.127140,7.863928e+04,...,125,0,0,1,1,0,0,0,0,Normal
125024,0.001058,udp,dns,CON,2,2,146,178,2835.538600,5.519849e+05,...,89,0,0,3,1,0,0,0,0,Normal
125025,0.613896,tcp,-,FIN,10,6,738,268,24.434107,8.665963e+03,...,45,0,0,1,1,0,0,0,0,Normal


In [ ]:
from sklearn.preprocessing import LabelEncoder

cat_cols = ['proto', 'service', 'state']
le = LabelEncoder()

train_data_copy1 = copy.deepcopy(majority_class_data)  # Create a deep copy of the DataFrame

for col in cat_cols:
    train_data_copy1[col] = le.fit_transform(train_data_copy1[col])

majority_class_data = copy.deepcopy(train_data_copy1)  # Update the original DataFrame with the modified copy

In [ ]:
majority_class_data

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,dmean,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,label
0,0.059893,113,0,3,60,62,3614,50036,2020.269400,4.747132e+05,...,807,0,0,1,1,0,0,0,0,Normal
1,0.002852,113,4,3,6,8,320,1826,4558.204600,7.489481e+05,...,228,0,0,1,1,0,0,0,0,Normal
2,1.195022,113,5,3,14,18,1684,10168,25.940945,1.047010e+04,...,565,1,3924,1,1,0,0,1,0,Normal
3,0.015786,113,0,3,48,50,2974,30506,6144.685500,1.476245e+06,...,610,0,0,1,1,0,0,0,0,Normal
4,0.024833,113,0,3,60,62,3598,48918,4872.549000,1.140096e+06,...,789,0,0,1,1,0,0,0,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125022,0.019016,113,0,3,48,50,2958,32642,5100.968000,1.218763e+06,...,653,0,0,1,1,0,0,0,0,Normal
125023,0.788817,113,0,3,122,126,7818,15800,313.127140,7.863928e+04,...,125,0,0,1,1,0,0,0,0,Normal
125024,0.001058,119,2,1,2,2,146,178,2835.538600,5.519849e+05,...,89,0,0,3,1,0,0,0,0,Normal
125025,0.613896,113,0,3,10,6,738,268,24.434107,8.665963e+03,...,45,0,0,1,1,0,0,0,0,Normal


In [ ]:
print("Number of attacks in dataset :",len(majority_class_data["label"].value_counts()))
majority_class_data["label"].value_counts()

Number of attacks in dataset : 3


Normal      79965
Exploits    26749
Fuzzers     18313
Name: label, dtype: int64

In [ ]:
majority_class_data

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,dmean,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,label
0,0.059893,113,0,3,60,62,3614,50036,2020.269400,4.747132e+05,...,807,0,0,1,1,0,0,0,0,Normal
1,0.002852,113,4,3,6,8,320,1826,4558.204600,7.489481e+05,...,228,0,0,1,1,0,0,0,0,Normal
2,1.195022,113,5,3,14,18,1684,10168,25.940945,1.047010e+04,...,565,1,3924,1,1,0,0,1,0,Normal
3,0.015786,113,0,3,48,50,2974,30506,6144.685500,1.476245e+06,...,610,0,0,1,1,0,0,0,0,Normal
4,0.024833,113,0,3,60,62,3598,48918,4872.549000,1.140096e+06,...,789,0,0,1,1,0,0,0,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125022,0.019016,113,0,3,48,50,2958,32642,5100.968000,1.218763e+06,...,653,0,0,1,1,0,0,0,0,Normal
125023,0.788817,113,0,3,122,126,7818,15800,313.127140,7.863928e+04,...,125,0,0,1,1,0,0,0,0,Normal
125024,0.001058,119,2,1,2,2,146,178,2835.538600,5.519849e+05,...,89,0,0,3,1,0,0,0,0,Normal
125025,0.613896,113,0,3,10,6,738,268,24.434107,8.665963e+03,...,45,0,0,1,1,0,0,0,0,Normal


In [ ]:
def preprocessing(data,cls):
    #----------attack categorization----------------->
    if cls=='multiclass':
        data['label']=data['label'].replace(['Normal'],0)
        # data['label']=data['label'].replace(['Backdoor'],1)
        # data['label']=data['label'].replace(['Analysis'],2)
        data['label']=data['label'].replace(['Fuzzers'],1)
        # data['label']=data['label'].replace(['Shellcode'],4)
        # data['label']=data['label'].replace(['Reconnaissance'],5)
        data['label']=data['label'].replace(['Exploits'],2)
        # data['label']=data['label'].replace(['DoS'],7)
        # data['label']=data['label'].replace(['Worms'],8)
        # data['label']=data['label'].replace(['Generic'],9)

#     cat_cols = ['proto', 'service', 'state']
#     le = LabelEncoder()
#     for col in cat_cols:
#         data[col] = le.fit_transform(data[col])

    #-------------------------splitting features and labels---------------->
    y=data['label']
    x=data.iloc[:, :-1]

    return x, y

In [ ]:
X_train, y_train = preprocessing(majority_class_data, "multiclass")

In [ ]:
y_train.value_counts()

0    79965
2    26749
1    18313
Name: label, dtype: int64

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

In [ ]:
len(X_train), len(y_train), #len(X_test), len(y_test)

(125027, 125027)

In [ ]:
# X_train[100020]

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [ ]:
X_train

array([[-0.25591813,  0.04219801, -0.67106516, ..., -0.13712667,
        -0.34874782, -0.06967358],
       [-0.26763408,  0.04219801,  0.7506528 , ..., -0.13712667,
        -0.34874782, -0.06967358],
       [-0.02276812,  0.04219801,  1.10608229, ..., -0.13712667,
         1.58203611, -0.06967358],
       ...,
       [-0.26800256,  0.57316764,  0.03979382, ..., -0.13712667,
        -0.34874782, -0.06967358],
       [-0.14212859,  0.04219801, -0.67106516, ..., -0.13712667,
        -0.34874782, -0.06967358],
       [-0.26821781,  0.57316764, -0.67106516, ..., -0.13712667,
        -0.34874782, -0.06967358]])

In [ ]:
y_train.value_counts()

0    79965
2    26749
1    18313
Name: label, dtype: int64

In [ ]:
model = svm.SVC(decision_function_shape='ovo', kernel='rbf')
model.fit(X_train, y_train)

SVC(decision_function_shape='ovo')

In [ ]:
model.classes_

array([0, 1, 2])

In [ ]:
all_support_vectors_per_class = {}

for class_label in model.classes_:
    class_mask = np.isin(model.support_, model.support_[model.predict(model.support_vectors_) == class_label])
    class_support_vectors = model.support_vectors_[class_mask]

    if len(class_support_vectors) > 0:
        # Store all support vectors for the current class
        all_support_vectors_per_class[class_label] = class_support_vectors
    else:
        # Handle the case where there are no support vectors for the class
        all_support_vectors_per_class[class_label] = None

In [ ]:
import pandas as pd

all_support_vectors_data = pd.DataFrame()

for class_label in model.classes_:
    class_mask = np.isin(model.support_, model.support_[model.predict(model.support_vectors_) == class_label])
    class_support_vectors = model.support_vectors_[class_mask]

    if len(class_support_vectors) > 0:

        majority_class_data_subset = majority_data_subset.drop(columns=['label'], errors='ignore')
        # Extract rows from majority_class_data_subset corresponding to support vectors
        support_vectors_data = majority_class_data_subset.loc[model.support_[class_mask]]

        support_vectors_data['label'] = class_label

        # Concatenate the support vectors data to the result DataFrame
        all_support_vectors_data = pd.concat([all_support_vectors_data, support_vectors_data], ignore_index=True)

<ipython-input-254-585474224fb7>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_support_vectors_data = all_support_vectors_data.append(support_vectors_data, ignore_index=False)
<ipython-input-254-585474224fb7>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_support_vectors_data = all_support_vectors_data.append(support_vectors_data, ignore_index=False)
<ipython-input-254-585474224fb7>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_support_vectors_data = all_support_vectors_data.append(support_vectors_data, ignore_index=False)


In [ ]:
import pandas as pd

class_labels_mapping = {0: 'Normal', 1: 'Fuzzers', 2: 'Exploits'}

all_support_vectors_data = pd.DataFrame()

for class_label in model.classes_:
    class_mask = np.isin(model.support_, model.support_[model.predict(model.support_vectors_) == class_label])
    class_support_vectors = model.support_vectors_[class_mask]

    if len(class_support_vectors) > 0:
        # Extract rows from majority_class_data_subset corresponding to support vectors
        support_vectors_data = majority_class_data_subset.loc[model.support_[class_mask]]

        support_vectors_data['class_label'] = class_labels_mapping[class_label]

        # Concatenate the support vectors data to the result DataFrame
        all_support_vectors_data = pd.concat([all_support_vectors_data, support_vectors_data], ignore_index=True)


In [ ]:
all_support_vectors_data.rename(columns={'class_label': 'label'}, inplace=True)

In [ ]:
all_support_vectors_data

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,dmean,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,label
0,1.573694,tcp,-,FIN,10,8,624,354,10.802608,2.856972e+03,...,44,0,0,1,1,0,0,0,0,Normal
1,0.965034,tcp,-,FIN,14,12,864,678,25.905823,6.656760e+03,...,57,0,0,1,1,0,0,0,0,Normal
2,1.292977,tcp,ftp,FIN,14,12,676,682,19.335224,3.885607e+03,...,57,0,0,1,1,0,0,0,0,Normal
3,0.411480,tcp,-,FIN,10,6,632,268,36.453777,1.106251e+04,...,45,0,0,1,1,0,0,0,0,Normal
4,0.463354,tcp,-,FIN,10,6,2516,268,32.372658,3.910617e+04,...,45,0,0,1,1,0,0,0,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49515,1.439615,tcp,http,FIN,10,10,836,2014,13.197973,4.184452e+03,...,201,1,542,1,1,0,0,1,0,Exploits
49516,20.815200,udp,dns,CON,184,182,17628,16380,17.535261,6.738537e+03,...,90,0,0,2,2,0,0,0,0,Exploits
49517,0.000009,gmtp,-,INT,2,0,200,0,111111.110000,8.888889e+07,...,0,0,0,2,2,0,0,0,0,Exploits
49518,0.352070,tcp,http,FIN,54,12,61987,526,184.622380,1.382452e+06,...,44,1,0,1,1,0,0,1,0,Exploits


In [ ]:
all_support_vectors_data['proto'].value_counts()

tcp          41659
udp           5468
ospf           330
unas           192
sctp           125
             ...  
ddx             10
tlsp            10
xtp             10
ipv6-opts        8
rtp              1
Name: proto, Length: 133, dtype: int64

In [ ]:
# all_samples_per_class = {}

# for class_label in model.classes_:
#     # Identify the support vectors for the current class
#     class_mask = np.isin(model.support_, model.support_[model.predict(model.support_vectors_) == class_label])
#     class_support_vectors = model.support_vectors_[class_mask]

#     # Identify all samples for the current class
#     all_samples_mask = (y_train == class_label)  # Assuming y_train contains class labels

#     # Extract all samples for the current class
#     all_samples = X_train[all_samples_mask]

#     if len(class_support_vectors) > 0:
#         # Find samples that are not support vectors
#         non_support_vector_indices = ~np.isin(np.arange(len(all_samples)), model.support_[class_mask])
#         non_support_vectors = all_samples[non_support_vector_indices]

#         # Store all samples and their corresponding support vectors for the current class
#         all_samples_per_class[class_label] = {'samples': all_samples, 'support_vectors': class_support_vectors, 'non_support_vectors': non_support_vectors}
#     else:
#         # Handle the case where there are no support vectors for the class
#         all_samples_per_class[class_label] = {'samples': all_samples, 'support_vectors': None, 'non_support_vectors': all_samples}

In [ ]:
# len(all_support_vectors_per_class[0])

31726

In [ ]:
# Get support vectors for each specified class closest to the decision boundary
closest_support_vectors = {}
for class_label in model.classes_:
    class_mask = np.isin(model.support_, model.support_[model.predict(model.support_vectors_) == class_label])
    class_support_vectors = model.support_vectors_[class_mask]

    if len(class_support_vectors) > 0:
        # Calculate the distance from each support vector to the decision boundary
        distances = model.decision_function(class_support_vectors)
        absolute_distances = np.abs(distances)

        # Find the index of the support vector with the smallest absolute distance for each support vector
        closest_support_vector_indices = np.argmin(absolute_distances, axis=1)

        # Use the first index (0) since we're interested in the absolute minimum
        closest_support_vector_index = closest_support_vector_indices[0]

        # Store the closest support vector for the current class
        closest_support_vectors[class_label] = class_support_vectors[closest_support_vector_index]
    else:
        # Handle the case where there are no support vectors for the class
        closest_support_vectors[class_label] = None

In [ ]:
print(np.sum(class_mask))
print(np.sum(~class_mask))

11142
38378


In [ ]:
# from sklearn.metrics.pairwise import euclidean_distances

# top_samples_per_class_with_distances = {}

# # Number of top samples to extract
# top_samples_count = 14523

# # Loop over classes to find top samples based on distance to the centroid
# for class_label, centroid in closest_support_vectors.items():
#     if centroid is not None:
#         # Calculate distances from all samples to the centroid
#         distances_to_centroid = euclidean_distances(X_train, centroid.reshape(1, -1))

#         # Get indices of samples sorted by distance to the centroid
#         sorted_indices = np.argsort(distances_to_centroid.flatten())

#         # Extract top N samples and their distances
#         top_indices = sorted_indices[:top_samples_count]
#         top_samples = majority_class_data.iloc[top_indices]
#         top_distances = distances_to_centroid.flatten()[top_indices]

#         # Store the results in the dictionary
#         top_samples_per_class_with_distances[class_label] = {'indices': top_indices, 'samples': top_samples, 'distances': top_distances}
#     else:
#         # Handle the case where there is no support vector for the class
#         top_samples_per_class_with_distances[class_label] = {'indices': [], 'samples': [], 'distances': []}

In [ ]:
top_samples_per_class_with_distances = {}

# Number of top samples to extract
top_samples_count = 14523

for class_label, centroid in closest_support_vectors.items():
    if centroid is not None:
        # Create a mask to select only samples of the current class
        class_mask = (y_train == class_label)

        print(f"centroid: {centroid}")
        print(np.sum(class_mask))
        print(np.sum(~class_mask))

        majority_class_data_subset = majority_data_subset.drop(columns=['label'], errors='ignore')

        # Convert NumPy array to DataFrame
        class_data_df = pd.DataFrame(X_train[class_mask], columns=majority_class_data_subset.columns)

        # Calculate distances from samples of the current class to the centroid
        distances_to_centroid = euclidean_distances(class_data_df, centroid.reshape(1, -1))

        # Get indices of samples sorted by distance to the centroid
        sorted_indices = np.argsort(distances_to_centroid.flatten())

        # Extract top N samples and their distances using .loc for boolean indexing
        top_indices = sorted_indices[:top_samples_count]
        top_samples = majority_data_subset.loc[class_mask].loc[class_data_df.index.isin(top_indices)]
        top_distances = distances_to_centroid.flatten()[top_indices]

        # Store the results in the dictionary
        top_samples_per_class_with_distances[class_label] = {'indices': top_indices, 'samples': top_samples, 'distances': top_distances}
    else:
        # Handle the case where there is no support vector for the class
        top_samples_per_class_with_distances[class_label] = {'indices': [], 'samples': [], 'distances': []}

centroid: [ 0.05500928  0.04219801 -0.67106516  0.22730914 -0.13557165 -0.23885215
 -0.06216148 -0.18406724 -0.20056684 -0.15982461 -0.40148061 -0.07907824
 -0.2266672  -0.06134286  0.06331212  0.0592391  -0.1212642   0.4679181
 -0.6827831  -0.69821692  0.47491166  1.85027809  1.62655865  1.79047504
 -0.47067888 -0.58250922 -0.16890589 -0.06355246 -0.34046793 -0.14228138
 -0.13714829 -0.13712667 -0.34874782 -0.06967358]
79965
45062
centroid: [-0.26821925  0.57316764 -0.67106516  1.47269828 -0.1824725  -0.31475194
 -0.06452958 -0.1866666   5.3794831   0.57697117 -0.40195415 -0.10332104
 -0.24715674 -0.0999345  -0.10174705 -0.13529958 -0.18815546 -2.13750643
 -1.25821652 -1.25618077 -2.10598033 -0.6643223  -0.5790227  -0.64888708
 -0.50879509 -0.72183635 -0.16890589 -0.06355246  1.99271107 -0.14228138
 -0.13714829 -0.13712667 -0.34874782 -0.06967358]
18313
106714
centroid: [-0.0649452   0.04219801  1.10608229  0.22730914 -0.13557165 -0.21987721
 -0.06122335 -0.17823702 -0.2004263  -0.159

In [ ]:
class_data_df

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,smean,dmean,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports
0,0.218651,0.042198,2.527800,0.227309,0.356887,-0.030128,0.409345,-0.170321,-0.199879,-0.157992,...,3.516276,-0.487513,-0.168906,-0.063552,-0.340468,-0.142281,-0.137148,-0.137127,-0.348748,-0.069674
1,0.483655,0.042198,2.527800,0.227309,0.778995,0.064747,0.821029,-0.167561,-0.199810,-0.157594,...,3.760220,-0.516012,-0.168906,-0.063552,-0.340468,-0.142281,-0.137148,-0.137127,-0.348748,-0.069674
2,-0.117996,0.042198,-0.671065,0.227309,-0.123846,-0.219877,-0.060777,-0.157560,-0.200267,-0.159790,...,-0.413505,0.532108,-0.168906,-0.063552,-0.340468,-0.142281,-0.137148,-0.137127,-0.348748,-0.069674
3,-0.152235,0.042198,1.816941,0.227309,-0.076946,-0.087053,-0.060586,-0.134694,-0.199473,-0.159770,...,-0.520230,0.212289,-0.168906,-0.063552,-0.340468,-0.142281,-0.137148,-0.137127,-0.348748,-0.069674
4,-0.121704,0.042198,1.106082,0.227309,-0.135572,-0.257827,-0.059857,-0.184699,-0.200396,-0.159779,...,-0.276286,-0.579343,-0.168906,-0.063552,0.242827,-0.142281,-0.137148,-0.137127,-0.348748,-0.069674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26744,-0.195906,0.042198,1.106082,0.227309,0.122383,-0.200902,0.217288,-0.182804,-0.197657,-0.152494,...,3.668741,-0.582509,0.842677,-0.063552,-0.340468,-0.142281,-0.137148,-0.137127,1.582036,-0.069674
26745,-0.268219,0.573168,-0.671065,1.472698,-0.182473,-0.314752,-0.054975,-0.186667,2.589368,7.640438,...,3.489595,-0.721836,-0.168906,-0.063552,-0.340468,-0.142281,-0.137148,-0.137127,-0.348748,-0.069674
26746,-0.167078,0.042198,1.106082,0.227309,-0.135572,-0.238852,-0.061205,-0.165270,-0.200170,-0.159775,...,-0.390635,0.430779,0.842677,-0.043345,-0.340468,-0.142281,-0.137148,-0.137127,1.582036,-0.069674
26747,-0.126753,0.042198,-0.671065,0.227309,-0.088671,-0.125002,-0.060431,-0.141934,-0.199848,-0.159781,...,-0.493549,0.243954,-0.168906,-0.063552,-0.340468,-0.142281,-0.137148,-0.137127,-0.348748,-0.069674


In [ ]:
ppp = (y_train == 0)

In [ ]:
len(ppp)

51011

In [ ]:
import numpy as np

ppp = (y_train == 2)

count_true = np.sum(ppp)

count_false = np.sum(~ppp)

print(f"Number of True values: {count_true}")
print(f"Number of False values: {count_false}")

Number of True values: 11618
Number of False values: 39393


In [ ]:
majority_data_subset.columns

Index(['dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt',
       'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt',
       'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_src_dport_ltm', 'ct_dst_sport_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'is_sm_ips_ports',
       'label'],
      dtype='object')

In [ ]:
for class_label, data_dict in top_samples_per_class_with_distances.items():
    print(f"Class {class_label}:")

    indices = data_dict['indices'][:10]
    samples = data_dict['samples'][:10]
    distances = data_dict['distances'][:10]

    for idx, sample, distance in zip(indices, samples.values, distances):
        print(f"  Index: {idx}, Distance: {distance}")
        print("  Sample:")
        print(sample)
        print("\n")

Class 0:
  Index: 2, Distance: 4.2146848510894035e-08
  Sample:
[0.646711 'tcp' '-' 'FIN' 10 6 534 268 23.194286 5950.1074 2770.944 2 1
 68.42956 116.9476 3578.428 195.45958 255 763588745 3117167491 255
 0.176056 0.061968 0.114088 53 45 0 0 1 1 0 0 0 0 'Normal']


  Index: 43304, Distance: 0.19986235237657363
  Sample:
[1.057639 'tcp' '-' 'FIN' 10 8 672 354 16.073538 4576.2305 2344.8455 2 1
 112.197334 138.42172 6046.003 245.05325 255 2034376110 982884080 255
 0.171422 0.088677 0.082745 67 44 0 0 2 2 0 0 0 0 'Normal']


  Index: 37057, Distance: 0.3065268205216763
  Sample:
[0.924435 'tcp' '-' 'FIN' 10 6 756 268 16.226128 5893.329 1938.4813 2 1
 99.211 173.8598 6234.135 303.918 255 2752926879 398326083 255 0.099914
 0.055134 0.04478 76 45 0 0 1 1 0 0 0 0 'Normal']


  Index: 28042, Distance: 0.3178628589584623
  Sample:
[0.544842 'tcp' '-' 'FIN' 10 8 604 354 31.201706 7987.6367 4551.7783 2 1
 58.013332 65.50514 3915.7344 111.561646 255 1283198448 998802442 255
 0.109646 0.086303 0.0233

In [ ]:
zero = top_samples_per_class_with_distances[2]['samples']

In [ ]:
majority_data_subset['proto'].value_counts()

tcp          102584
udp           19276
arp             720
ospf            400
unas            192
              ...  
tlsp             10
aris             10
xtp              10
ipv6-opts         8
rtp               1
Name: proto, Length: 133, dtype: int64

In [ ]:
majority_data_subset[:21]

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,dmean,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,label
0,2.095192,tcp,smtp,FIN,160,40,185946,2680,94.979360,7.055544e+05,...,67,0,0,1,1,0,0,0,0,Exploits
1,0.722247,tcp,-,FIN,10,6,590,268,20.768518,5.881645e+03,...,45,0,0,1,1,0,0,0,0,Fuzzers
2,1.073097,tcp,-,FIN,16,18,1540,1644,30.752113,1.076510e+04,...,91,0,0,1,1,0,0,0,0,Normal
3,0.001791,udp,-,CON,4,4,512,304,3908.431000,1.715243e+06,...,76,0,0,1,1,0,0,0,0,Normal
4,3.359477,tcp,ftp,FIN,24,22,1256,1668,13.394942,2.867113e+03,...,76,0,0,1,1,1,1,0,0,Exploits
5,0.646711,tcp,-,FIN,10,6,534,268,23.194286,5.950107e+03,...,45,0,0,1,1,0,0,0,0,Normal
6,0.976205,tcp,smtp,FIN,30,15,27790,1308,45.072502,2.201505e+05,...,87,0,0,1,1,0,0,0,0,Exploits
7,0.000009,udp,-,INT,2,0,556,0,111111.110000,2.471111e+08,...,0,0,0,15,1,0,0,0,0,Fuzzers
8,0.001789,udp,-,CON,4,4,528,304,3912.800500,1.770822e+06,...,76,0,0,1,1,0,0,0,0,Normal
9,0.355215,tcp,http,FIN,10,6,1016,268,42.227943,2.060724e+04,...,45,1,0,1,1,0,0,1,0,Exploits


In [ ]:
print("Number of attacks in dataset :",len(zero["label"].value_counts()))
zero["label"].value_counts()

Number of attacks in dataset : 1


Exploits    14523
Name: label, dtype: int64

In [ ]:
# for class_label, centroid in closest_support_vectors.items():
#   class_indices = np.where(y_train == class_label)[0]
#   print(class_indices)

[    0     1     2 ... 79962 79963 79964]
[106714 106715 106716 ... 125024 125025 125026]
[ 79965  79966  79967 ... 106711 106712 106713]


In [ ]:
# from sklearn.metrics.pairwise import euclidean_distances
# from sklearn import svm
# import numpy as np
# import pandas as pd

# # Number of top samples to extract
# top_samples_count = 14523

# # Fit SVM model
# model = svm.SVC(decision_function_shape='ovr', kernel='rbf')
# model.fit(X_train, y_train)

# # Get support vectors and their indices for each class
# support_vectors_per_class = {}
# for class_label in np.unique(y_train):
#     support_vectors_mask = model.support_[model.support_vectors_[:, -1] == class_label]
#     support_vectors_per_class[class_label] = X_train[support_vectors_mask]

# # Dictionary to store top samples for each class along with distances
# top_samples_per_class_with_distances = {}

# # Loop over classes to find top samples based on distance to the centroid
# for class_label, support_vectors in support_vectors_per_class.items():
#     if len(support_vectors) > 0:
#         # Calculate the centroid for the support vectors of the current class
#         centroid = np.mean(support_vectors, axis=0)

#         # Calculate distances from all samples to the centroid
#         distances_to_centroid = euclidean_distances(X_train, centroid.reshape(1, -1))

#         # Get indices of samples sorted by distance to the centroid
#         sorted_indices = np.argsort(distances_to_centroid.flatten())

#         # Extract top N samples and their distances
#         top_indices = sorted_indices[:top_samples_count]
#         top_samples = train_data.iloc[top_indices]
#         top_distances = distances_to_centroid.flatten()[top_indices]

#         # Store the results in the dictionary
#         top_samples_per_class_with_distances[class_label] = {'indices': top_indices, 'samples': top_samples, 'distances': top_distances}
#     else:
#         # Handle the case where there is no support vector for the class
#         top_samples_per_class_with_distances[class_label] = {'indices': [], 'samples': [], 'distances': []}

In [ ]:
zero = top_samples_per_class_with_distances[0]['samples']
three = top_samples_per_class_with_distances[1]['samples']
six = top_samples_per_class_with_distances[2]['samples']

In [ ]:
final_majority_class_data = pd.concat([zero, three, six], axis=0)

In [ ]:
final_majority_class_data.reset_index(drop=True, inplace=True)

In [ ]:
final_majority_class_data

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,dmean,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,label
0,0.646711,tcp,-,FIN,10,6,534,268,23.194286,5950.1074,...,45,0,0,1,1,0,0,0,0,Normal
1,1.057639,tcp,-,FIN,10,8,672,354,16.073538,4576.2305,...,44,0,0,2,2,0,0,0,0,Normal
2,0.924435,tcp,-,FIN,10,6,756,268,16.226128,5893.3290,...,45,0,0,1,1,0,0,0,0,Normal
3,0.544842,tcp,-,FIN,10,8,604,354,31.201706,7987.6367,...,44,0,0,2,1,0,0,0,0,Normal
4,1.791973,tcp,ftp,FIN,16,12,782,682,15.067191,3276.8352,...,57,0,0,1,1,0,0,0,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43564,1.995611,tcp,ftp,FIN,18,16,1072,1054,16.536290,4060.9119,...,66,0,0,1,1,0,0,0,0,Exploits
43565,0.237725,tcp,http,FIN,10,6,1308,268,63.098118,39642.4400,...,45,1,0,1,1,0,0,1,0,Exploits
43566,3.119221,tcp,ftp,FIN,24,22,1276,1616,14.426679,3136.6807,...,73,0,0,1,1,0,0,0,0,Exploits
43567,0.825808,tcp,http,FIN,10,8,926,354,20.585900,8079.3600,...,44,0,0,1,1,0,0,0,0,Exploits


In [ ]:
print("Number of attacks in dataset :",len(final_majority_class_data["label"].value_counts()))
final_majority_class_data["label"].value_counts()

Number of attacks in dataset : 3


Normal      14523
Fuzzers     14523
Exploits    14523
Name: label, dtype: int64

In [ ]:
# Reconnaissance = train_data[train_data['label'] == 'Reconnaissance']
# DoS = train_data[train_data['label'] == 'DoS']
# Generic = train_data[train_data['label'] == 'Generic']
# Analysis = train_data[train_data['label'] == 'Analysis']
# Backdoor = train_data[train_data['label'] == 'Backdoor']
# Shellcode = train_data[train_data['label'] == 'Shellcode']
# Worms = train_data[train_data['label'] == 'Worms']

In [ ]:
final_minority_class_data = pd.concat([Reconnaissance, DoS, Generic, Analysis, Backdoor, Shellcode, Worms], axis=0)

In [ ]:
# train_data = pd.concat([final_minority_class_data, final_majority_class_data], axis=0)
train_data = pd.concat([final_minority_class_data, all_support_vectors_data], axis=0)

In [ ]:
print("Number of attacks in dataset :",len(train_data["label"].value_counts()))
train_data["label"].value_counts()

Number of attacks in dataset : 10


Normal            31721
Exploits          11142
Reconnaissance     8012
Fuzzers            6657
DoS                4873
Generic            3057
Analysis           1442
Backdoor           1389
Shellcode          1257
Worms               165
Name: label, dtype: int64

In [ ]:
train_data

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,dmean,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,label
243,0.921987,ospf,-,INT,20,0,1280,0,20.607666,1.055112e+04,...,0,0,0,1,1,0,0,0,0,Reconnaissance
256,0.674665,tcp,-,FIN,10,8,564,354,25.197693,6.023730e+03,...,44,0,0,1,1,0,0,0,0,Reconnaissance
259,0.000003,udp,-,INT,2,0,168,0,333333.312500,2.240000e+08,...,0,0,0,1,1,0,0,0,0,Reconnaissance
260,0.663844,tcp,-,FIN,10,8,564,354,25.608425,6.121920e+03,...,44,0,0,1,1,0,0,0,0,Reconnaissance
261,0.217312,tcp,http,FIN,10,6,1148,268,69.025185,3.806509e+04,...,45,1,0,1,1,0,0,1,0,Reconnaissance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49515,1.439615,tcp,http,FIN,10,10,836,2014,13.197973,4.184452e+03,...,201,1,542,1,1,0,0,1,0,Exploits
49516,20.815200,udp,dns,CON,184,182,17628,16380,17.535261,6.738537e+03,...,90,0,0,2,2,0,0,0,0,Exploits
49517,0.000009,gmtp,-,INT,2,0,200,0,111111.110000,8.888889e+07,...,0,0,0,2,2,0,0,0,0,Exploits
49518,0.352070,tcp,http,FIN,54,12,61987,526,184.622380,1.382452e+06,...,44,1,0,1,1,0,0,1,0,Exploits


In [ ]:
train_data = train_data.drop_duplicates()
train_data = train_data.dropna()
train_data = train_data.replace([np.inf, -np.inf], np.nan)

In [ ]:
train_data

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,dmean,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,label
243,0.921987,ospf,-,INT,20,0,1280,0,20.607666,1.055112e+04,...,0,0,0,1,1,0,0,0,0,Reconnaissance
256,0.674665,tcp,-,FIN,10,8,564,354,25.197693,6.023730e+03,...,44,0,0,1,1,0,0,0,0,Reconnaissance
259,0.000003,udp,-,INT,2,0,168,0,333333.312500,2.240000e+08,...,0,0,0,1,1,0,0,0,0,Reconnaissance
260,0.663844,tcp,-,FIN,10,8,564,354,25.608425,6.121920e+03,...,44,0,0,1,1,0,0,0,0,Reconnaissance
261,0.217312,tcp,http,FIN,10,6,1148,268,69.025185,3.806509e+04,...,45,1,0,1,1,0,0,1,0,Reconnaissance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49514,0.539749,tcp,http,FIN,10,6,950,268,27.790693,1.267256e+04,...,45,0,0,1,1,0,0,0,0,Exploits
49515,1.439615,tcp,http,FIN,10,10,836,2014,13.197973,4.184452e+03,...,201,1,542,1,1,0,0,1,0,Exploits
49516,20.815200,udp,dns,CON,184,182,17628,16380,17.535261,6.738537e+03,...,90,0,0,2,2,0,0,0,0,Exploits
49518,0.352070,tcp,http,FIN,54,12,61987,526,184.622380,1.382452e+06,...,44,1,0,1,1,0,0,1,0,Exploits


In [ ]:
cat_cols = ["proto", "service", "state"]
le = LabelEncoder()
for col in cat_cols:
    train_data[col] = le.fit_transform(train_data[col])

# cat_cols = ["service"]
# le = LabelEncoder()
# for col in cat_cols:
#     train_data[col] = le.fit_transform(train_data[col])

In [ ]:
def preprocessing(data,cls,df):
    #----------attack categorization----------------->
    if cls=='multiclass':
        data['label']=data['label'].replace(['Normal'],0)
        data['label']=data['label'].replace(['Generic'],1)
        data['label']=data['label'].replace(['Exploits'],2)
        data['label']=data['label'].replace(['Fuzzers'],3)
        data['label']=data['label'].replace(['DoS'],4)
        data['label']=data['label'].replace(['Reconnaissance'],5)
        data['label']=data['label'].replace(['Analysis'],6)
        data['label']=data['label'].replace(['Backdoor'],7)
        data['label']=data['label'].replace(['Shellcode'],8)
        data['label']=data['label'].replace(['Worms'],9)


    #-------------------------splitting features and labels---------------->
    y=data['label']
    x=data.drop('label', axis=1)
    x_sub = copy.deepcopy(x)

    #-----------------converting to binary label vectors-------------------------------------->
    yt=y.copy()
#     yt=pd.get_dummies(yt)
    #------------------------------------------------------------------------------>

#     x = MinMaxScaler(feature_range=(0, 1)).fit_transform(x)
    x = StandardScaler().fit_transform(x)
    x = pd.DataFrame(x, columns=x_sub.columns)

    return x,yt

In [ ]:
x_train,Y_train=preprocessing(train_data,cls='multiclass',df='train')

In [ ]:
x_train

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,smean,dmean,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports
0,-0.114845,-2.071571,-0.587285,1.619612,-0.014871,-0.097664,-0.053222,-0.049337,-0.323684,-0.250976,...,-0.480227,-0.512106,-0.146124,-0.028299,-0.272684,-0.206925,-0.095895,-0.095994,-0.307088,-0.012702
1,-0.157346,0.168786,-0.587285,-0.324054,-0.061875,-0.051807,-0.055784,-0.047799,-0.323642,-0.250994,...,-0.508376,-0.228347,-0.146124,-0.028299,-0.272684,-0.206925,-0.095895,-0.095994,-0.307088,-0.012702
2,-0.273284,0.552847,-0.587285,1.619612,-0.099479,-0.097664,-0.057201,-0.049337,2.741301,0.640996,...,-0.409855,-0.512106,-0.146124,-0.028299,-0.272684,-0.206925,-0.095895,-0.095994,-0.307088,-0.012702
3,-0.159206,0.168786,-0.587285,-0.324054,-0.061875,-0.051807,-0.055784,-0.047799,-0.323639,-0.250993,...,-0.508376,-0.228347,-0.146124,-0.028299,-0.272684,-0.206925,-0.095895,-0.095994,-0.307088,-0.012702
4,-0.235940,0.168786,1.631172,-0.324054,-0.061875,-0.063271,-0.053694,-0.048172,-0.323239,-0.250866,...,-0.300777,-0.221898,0.623019,-0.028299,-0.272684,-0.206925,-0.095895,-0.095994,1.145179,-0.012702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68180,-0.180531,0.168786,1.631172,-0.324054,-0.061875,-0.063271,-0.054403,-0.048172,-0.323618,-0.250967,...,-0.371150,-0.221898,-0.146124,-0.028299,-0.272684,-0.206925,-0.095895,-0.095994,-0.307088,-0.012702
68181,-0.025893,0.168786,1.631172,-0.324054,-0.061875,-0.040343,-0.054811,-0.040583,-0.323753,-0.251001,...,-0.409855,0.784156,0.623019,-0.021810,-0.272684,-0.206925,-0.095895,-0.095994,1.145179,-0.012702
68182,3.303708,0.552847,0.300098,-4.211385,0.756005,0.945582,0.005268,0.021867,-0.323713,-0.250991,...,-0.367631,0.068310,-0.146124,-0.028299,-0.009171,0.278845,-0.095895,-0.095994,-0.307088,-0.012702
68183,-0.212783,0.168786,1.631172,-0.324054,0.144945,-0.028879,0.163977,-0.047051,-0.322176,-0.245512,...,3.333959,-0.228347,0.623019,-0.028299,-0.272684,-0.206925,-0.095895,-0.095994,1.145179,-0.012702


In [ ]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif
from math import log2

information_gains = mutual_info_classif(x_train, Y_train)

# Calculate split information
split_info = -sum((x / len(x_train)) * log2(x / len(x_train)) for x in x_train.nunique())

# Calculate information gain ratio
information_gain_ratios = information_gains / split_info

# Step 5: Sort the results in descending order
results = sorted(zip(x_train.columns, information_gains, information_gain_ratios),
                 key=lambda x: x[1], reverse=True)

# Step 6: Print the sorted results
for feature, info_gain, info_gain_ratio in results:
    print(f"Feature: {feature}")
    print(f"Information Gain: {info_gain}")
    print(f"Information Gain Ratio: {info_gain_ratio}")
    print()

Feature: sbytes
Information Gain: 0.8912105433048942
Information Gain Ratio: 0.24265279146574043

Feature: dbytes
Information Gain: 0.6339336802239015
Information Gain Ratio: 0.1726031836877114

Feature: smean
Information Gain: 0.6286524495707857
Information Gain Ratio: 0.17116524585138293

Feature: dmean
Information Gain: 0.5375156615797456
Information Gain Ratio: 0.1463511363489984

Feature: dloss
Information Gain: 0.37743825683351195
Information Gain Ratio: 0.10276634103427779

Feature: spkts
Information Gain: 0.36003412694053205
Information Gain Ratio: 0.09802766201696828

Feature: dload
Information Gain: 0.3564770948815634
Information Gain Ratio: 0.09705917733629885

Feature: sloss
Information Gain: 0.34569541959007744
Information Gain Ratio: 0.09412361555933137

Feature: dpkts
Information Gain: 0.33684415006665036
Information Gain Ratio: 0.09171365163553112

Feature: sload
Information Gain: 0.32813353994705174
Information Gain Ratio: 0.08934198550481837

Feature: synack
Informati

In [ ]:
# from sklearn.preprocessing import LabelEncoder

# cat_cols = ['proto', 'service', 'state']
# le = LabelEncoder()

# train_data_copy1 = copy.deepcopy(train_data)  # Create a deep copy of the DataFrame

# for col in cat_cols:
#     train_data_copy1[col] = le.fit_transform(train_data_copy1[col])

# train_data = copy.deepcopy(train_data_copy1)  # Update the original DataFrame with the modified copy

In [ ]:
train_data

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,dmean,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,label
243,0.921987,75,0,4,20,0,1280,0,20.607666,1.055112e+04,...,0,0,0,1,1,0,0,0,0,5
256,0.674665,109,0,3,10,8,564,354,25.197693,6.023730e+03,...,44,0,0,1,1,0,0,0,0,5
259,0.000003,115,0,4,2,0,168,0,333333.312500,2.240000e+08,...,0,0,0,1,1,0,0,0,0,5
260,0.663844,109,0,3,10,8,564,354,25.608425,6.121920e+03,...,44,0,0,1,1,0,0,0,0,5
261,0.217312,109,5,3,10,6,1148,268,69.025185,3.806509e+04,...,45,1,0,1,1,0,0,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43564,0.185435,109,5,3,10,6,1402,268,80.890880,5.444495e+04,...,45,1,0,1,1,0,0,1,0,2
43565,0.448119,109,5,3,10,6,1314,268,33.473250,2.111939e+04,...,45,1,0,1,1,0,0,1,0,2
43566,1.255574,109,7,3,24,48,1148,42124,56.547840,7.015118e+03,...,878,0,0,1,1,0,0,0,0,2
43567,0.577332,109,5,3,10,14,796,9922,39.838430,9.935357e+03,...,709,1,4437,1,1,0,0,1,0,2


In [ ]:
# train_data = train_data[['sbytes', 'dbytes', 'smean', 'dmean', 'dloss', 'spkts', 'dload','sloss', 'dpkts', 'sload', 'synack', 'tcprtt', 'service', 'djit',
#                          'dinpkt', 'rate', 'sjit', 'ackdat', 'dur', 'sinpkt', 'proto', 'label']]

train_data = train_data[['sbytes', 'dbytes', 'smean', 'dmean', 'dloss', 'spkts', 'dload','sloss', 'dpkts', 'sload', 'synack', 'tcprtt', 'service', 'djit',
                         'label']]

In [ ]:
train_data = pd.get_dummies(train_data, columns=['proto', 'service', 'state'])
# train_data = pd.get_dummies(train_data, columns=['service', 'proto'])

In [ ]:
train_data = shuffle(train_data)

# If you want to reset the index after shuffling
train_data.reset_index(drop=True, inplace=True)

In [ ]:
train_data['label'].value_counts()

Normal            31644
Exploits          10491
Reconnaissance     8012
Fuzzers            5855
DoS                4873
Generic            3057
Analysis           1442
Backdoor           1389
Shellcode          1257
Worms               165
Name: label, dtype: int64

In [ ]:
train_data

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sload,...,dmean,trans_depth,response_body_len,ct_src_dport_ltm,ct_dst_sport_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,is_sm_ips_ports,label
0,1.073995,113,0,4,10,8,804,354,15.828752,5.392949e+03,...,44,0,0,1,1,0,0,0,0,Normal
1,1.453971,113,0,4,10,8,504,436,11.692117,2.497987e+03,...,55,0,0,1,1,0,0,0,0,DoS
2,0.252468,113,5,4,10,8,802,1088,67.335266,2.287815e+04,...,136,1,117,1,1,0,0,1,0,DoS
3,0.597103,113,5,4,10,8,830,1250,28.470800,1.000832e+04,...,156,1,218,3,1,0,0,1,0,Normal
4,0.800498,113,0,4,14,10,774,556,28.732115,7.185527e+03,...,56,0,0,1,1,0,0,0,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68180,0.000036,119,2,5,2,0,114,0,27777.777344,1.266667e+07,...,0,0,0,1,1,0,0,0,0,Generic
68181,1.656119,113,0,4,10,8,564,354,10.264963,2.453930e+03,...,44,0,0,1,1,0,0,0,0,Reconnaissance
68182,1.209867,113,5,4,10,8,824,354,14.051131,4.906324e+03,...,44,1,0,1,1,0,0,1,0,Reconnaissance
68183,4.151655,113,5,4,338,746,18058,1087890,260.859800,3.469460e+04,...,1458,1,524288,2,1,0,0,1,0,Normal


In [ ]:
def preprocessing(data,cls,df):
    #----------attack categorization----------------->
    if cls=='multiclass':
        data['label']=data['label'].replace(['Normal'],0)
        data['label']=data['label'].replace(['Generic'],1)
        data['label']=data['label'].replace(['Exploits'],2)
        data['label']=data['label'].replace(['Fuzzers'],3)
        data['label']=data['label'].replace(['DoS'],4)
        data['label']=data['label'].replace(['Reconnaissance'],5)
        data['label']=data['label'].replace(['Analysis'],6)
        data['label']=data['label'].replace(['Backdoor'],7)
        data['label']=data['label'].replace(['Shellcode'],8)
        data['label']=data['label'].replace(['Worms'],9)


    #-------------------------splitting features and labels---------------->
    y=data['label']
    x=data.drop('label', axis=1)

    #-----------------converting to binary label vectors-------------------------------------->
    yt=y.copy()
#     yt=pd.get_dummies(yt)
    #------------------------------------------------------------------------------>

#     x = MinMaxScaler(feature_range=(0, 1)).fit_transform(x)
    x = StandardScaler().fit_transform(x)

    return x,yt

In [ ]:
x_train,Y_train=preprocessing(train_data,cls='multiclass',df='train')

In [ ]:
len(x_train)

68185

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_train, Y_train, test_size=0.2, random_state=42, stratify=Y_train)

In [ ]:
len(x_train), len(y_train), len(x_test), len(y_test)

(54548, 54548, 13637, 13637)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create an instance of DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(random_state=42)

# Train the decision tree
decision_tree.fit(x_train, y_train)

y_pred = decision_tree.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the evaluation metrics
print("Accuracy:", accuracy*100)
print("Precision:", precision*100)
print("Recall:", recall*100)
print("F1-score:", f1*100)

Accuracy: 78.72699274033879
Precision: 78.23334350832675
Recall: 78.72699274033879
F1-score: 78.23265035985321


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest classifier
clf = RandomForestClassifier(random_state=42)

# Train the classifier
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

In [ ]:

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the evaluation metrics
print("Accuracy:", accuracy*100)
print("Precision:", precision*100)
print("Recall:", recall*100)
print("F1-score:", f1*100)

Accuracy: 81.55019432426487
Precision: 80.1993214910097
Recall: 81.55019432426487
F1-score: 80.6780848073072
